In [1]:
import functools
from typing import Literal, Optional

import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import numpy as np
import optax
import scanpy as sc
from ott.neural import datasets
from ott.neural.methods.flows import dynamics, otfm
from ott.neural.networks.layers import time_encoder
from ott.neural.networks.velocity_field import VelocityField
from ott.solvers import utils as solver_utils
from torch.utils.data import DataLoader
from tqdm import tqdm

from ot_pert.metrics import compute_mean_metrics, compute_metrics_fast
from ot_pert.utils import ConditionalLoader

In [2]:
def reconstruct_data(embedding: np.ndarray, projection_matrix: np.ndarray, mean_to_add: np.ndarray) -> np.ndarray:
    return np.matmul(embedding, projection_matrix.T) + mean_to_add

In [3]:
obsm_key_cond = "ecfp_logdose_cell_line_new"
obsm_key_data = "X_pca"

In [4]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_30.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_30.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_30.h5ad"

In [8]:
adata_train = sc.read(adata_train_path)
adata_ood = sc.read(adata_ood_path)
adata_test = sc.read(adata_test_path)

/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/icb/dominik.klein/mambaforge/envs/ot_pert_genot/lib/python3.12/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [9]:
sc.pp.subsample(adata_train, fraction=0.5)

In [10]:
adata_train.obsm["ecfp_logdose_cell_line_new"] = np.concatenate(
    (adata_train.obsm["ecfp"], np.tile(np.asarray(adata_train.obs["logdose"])[:, None], (1, 100))), axis=1
)

adata_test.obsm["ecfp_logdose_cell_line_new"] = np.concatenate(
    (adata_test.obsm["ecfp"], np.tile(np.asarray(adata_test.obs["logdose"])[:, None], (1, 100))), axis=1
)

adata_ood.obsm["ecfp_logdose_cell_line_new"] = np.concatenate(
    (adata_ood.obsm["ecfp"], np.tile(np.asarray(adata_ood.obs["logdose"])[:, None], (1, 100))), axis=1
)

In [11]:
def data_match_fn(
    src_lin: Optional[jnp.ndarray],
    tgt_lin: Optional[jnp.ndarray],
    src_quad: Optional[jnp.ndarray],
    tgt_quad: Optional[jnp.ndarray],
    *,
    typ: Literal["lin", "quad", "fused"],
    epsilon: float = 1e-2,
    tau_a: float = 1.0,
    tau_b: float = 1.0,
) -> jnp.ndarray:
    if typ == "lin":
        return solver_utils.match_linear(
            x=src_lin, y=tgt_lin, scale_cost="mean", epsilon=epsilon, tau_a=tau_a, tau_b=tau_b
        )
    if typ == "quad":
        return solver_utils.match_quadratic(xx=src_quad, yy=tgt_quad)
    if typ == "fused":
        return solver_utils.match_quadratic(xx=src_quad, yy=tgt_quad, x=src_lin, y=tgt_lin)
    raise NotImplementedError(f"Unknown type: {typ}.")


# Load data

dls = []

train_data_source = {}
train_data_target = {}
train_data_source_decoded = {}
train_data_target_decoded = {}
train_data_conditions = {}


for cond in adata_train.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    src_str = list(adata_train[adata_train.obs["condition"] == cond].obs["cell_type"].unique())
    assert len(src_str) == 1
    source = adata_train[adata_train.obs["condition"] == src_str[0] + "_Vehicle_0.0"].obsm[obsm_key_data]
    source_decoded = adata_train[adata_train.obs["condition"] == src_str[0] + "_Vehicle_0.0"].X.A
    target = adata_train[adata_train.obs["condition"] == cond].obsm[obsm_key_data]
    target_decoded = adata_train[adata_train.obs["condition"] == cond].X.A
    conds = adata_train[adata_train.obs["condition"] == cond].obsm[obsm_key_cond]
    assert np.all(np.all(conds == conds[0], axis=1))
    conds = np.tile(conds[0], (len(source), 1))
    dls.append(
        DataLoader(
            datasets.OTDataset(
                datasets.OTData(
                    lin=source,
                    condition=conds,
                ),
                datasets.OTData(lin=target),
            ),
            batch_size=10,
            shuffle=True,
        )
    )
    train_data_source[cond] = source
    train_data_target[cond] = target
    train_data_conditions[cond] = conds
    train_data_source_decoded[cond] = source_decoded
    train_data_target_decoded[cond] = target_decoded

train_loader = ConditionalLoader(dls, seed=0)

test_data_source = {}
test_data_target = {}
test_data_source_decoded = {}
test_data_target_decoded = {}
test_data_conditions = {}

for cond in adata_test.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    src_str = list(adata_test[adata_test.obs["condition"] == cond].obs["cell_type"].unique())
    assert len(src_str) == 1
    source = adata_test[adata_test.obs["condition"] == src_str[0] + "_Vehicle_0.0"].obsm[obsm_key_data]
    source_decoded = adata_test[adata_test.obs["condition"] == src_str[0] + "_Vehicle_0.0"].X.A

    target = adata_test[adata_test.obs["condition"] == cond].obsm[obsm_key_data]
    target_decoded = adata_test[adata_test.obs["condition"] == cond].X.A
    conds = adata_test[adata_test.obs["condition"] == cond].obsm[obsm_key_cond]
    assert np.all(np.all(conds == conds[0], axis=1))
    conds = np.tile(conds[0], (len(source), 1))
    test_data_source[cond] = source
    test_data_target[cond] = target
    test_data_source_decoded[cond] = source_decoded
    test_data_target_decoded[cond] = target_decoded
    test_data_conditions[cond] = conds

ood_data_source = {}
ood_data_target = {}
ood_data_source_decoded = {}
ood_data_target_decoded = {}
ood_data_conditions = {}

for cond in adata_ood.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    src_str = list(adata_ood[adata_ood.obs["condition"] == cond].obs["cell_type"].unique())
    assert len(src_str) == 1
    source = adata_ood[adata_ood.obs["condition"] == src_str[0] + "_Vehicle_0.0"].obsm[obsm_key_data]
    source_decoded = adata_ood[adata_ood.obs["condition"] == src_str[0] + "_Vehicle_0.0"].X.A
    conds = adata_ood[adata_ood.obs["condition"] == cond].obsm[obsm_key_cond]
    assert np.all(np.all(conds == conds[0], axis=1))
    conds = np.tile(conds[0], (len(source), 1))
    ood_data_source[cond] = source
    ood_data_target[cond] = target
    ood_data_source_decoded[cond] = source_decoded
    ood_data_target_decoded[cond] = target_decoded
    ood_data_conditions[cond] = conds

reconstruct_data_fn = functools.partial(
    reconstruct_data, projection_matrix=adata_train.varm["PCs"], mean_to_add=adata_train.varm["X_train_mean"].T
)

2024-04-19 12:04:11.258610: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [12]:
reconstruct_data_fn = functools.partial(
    reconstruct_data, projection_matrix=adata_train.varm["PCs"], mean_to_add=adata_train.varm["X_train_mean"].T
)

train_deg_dict = {
    k: v for k, v in adata_train.uns["rank_genes_groups_cov_all"].items() if k in train_data_conditions.keys()
}
test_deg_dict = {
    k: v for k, v in adata_train.uns["rank_genes_groups_cov_all"].items() if k in test_data_conditions.keys()
}
ood_deg_dict = {
    k: v for k, v in adata_train.uns["rank_genes_groups_cov_all"].items() if k in ood_data_conditions.keys()
}


def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]

In [13]:
source_dim = source.shape[1]
target_dim = source_dim
condition_dim = conds.shape[1]

source_dim = source.shape[1]
target_dim = source_dim
condition_dim = conds.shape[1]

In [14]:
vf = VelocityField(
    hidden_dims=[1024, 1024, 1024],
    time_dims=[512, 512],
    output_dims=[1024, 1024, 1024] + [target_dim],
    condition_dims=[4096, 2048, 1024, 512, 256],
    time_encoder=functools.partial(time_encoder.cyclical_time_encoder, n_freqs=1024),
)

model = otfm.OTFlowMatching(
    vf,
    flow=dynamics.ConstantNoiseFlow(0),
    match_fn=jax.jit(
        functools.partial(data_match_fn, typ="lin", src_quad=None, tgt_quad=None, epsilon=0.1, tau_a=1.0, tau_b=1.0)
    ),
    condition_dim=condition_dim,
    rng=jax.random.PRNGKey(13),
    optimizer=optax.MultiSteps(optax.adam(learning_rate=1e-4), 50),
)

training_logs = {"loss": []}

In [15]:
rng = jax.random.PRNGKey(0)
for it in tqdm(range(100000)):
    rng, rng_resample, rng_step_fn = jax.random.split(rng, 3)
    batch = next(train_loader)
    batch = jtu.tree_map(jnp.asarray, batch)

    src, tgt = batch["src_lin"], batch["tgt_lin"]
    src_cond = batch.get("src_condition")

    if model.match_fn is not None:
        tmat = model.match_fn(src, tgt)
        src_ixs, tgt_ixs = solver_utils.sample_joint(rng_resample, tmat)
        src, tgt = src[src_ixs], tgt[tgt_ixs]
        src_cond = None if src_cond is None else src_cond[src_ixs]

    model.vf_state, loss = model.step_fn(
        rng_step_fn,
        model.vf_state,
        src,
        tgt,
        src_cond,
    )

    training_logs["loss"].append(float(loss))
    if (it % 100000 == 0) and (it > 0):
        idcs = np.random.choice(list(test_data_source.keys()), 20)
        test_data_source_tmp = {k: v for k, v in test_data_source.items() if k in idcs}
        test_data_target_tmp = {k: v for k, v in test_data_target.items() if k in idcs}
        test_data_conditions_tmp = {k: v for k, v in test_data_conditions.items() if k in idcs}
        test_data_target_decoded_tmp = {k: v for k, v in test_data_target_decoded.items() if k in idcs}
        test_deg_dict_tmp = {k: v for k, v in test_deg_dict.items() if k in idcs}
        valid_losses = []
        # for cond in test_data_source_tmp.keys():
        #    src = test_data_source_tmp[cond]
        #    tgt = test_data_target_tmp[cond]
        #    src_cond = test_data_conditions_tmp[cond]
        #    if model.match_fn is not None:
        #        tmat = model.match_fn(src, tgt)
        #        src_ixs, tgt_ixs = solver_utils.sample_joint(rng_resample, tmat)
        #        src, tgt = src[src_ixs], tgt[tgt_ixs]
        #        src_cond = None if src_cond is None else src_cond[src_ixs]
        #    _, valid_loss = model.step_fn(
        #        rng,
        #        model.vf_state,
        #        src,
        #        tgt,
        #        src_cond,
        #    )
        #    valid_losses.append(valid_loss)

        # predicted_target_train = jax.tree_util.tree_map(model.transport, train_data_source, train_data_conditions)
        # train_metrics = jax.tree_util.tree_map(compute_metrics_fast, train_data_target, predicted_target_train)
        # mean_train_metrics = compute_mean_metrics(train_metrics, prefix="train_")

        # predicted_target_train_decoded = jax.tree_util.tree_map(reconstruct_data_fn, predicted_target_train)
        # train_metrics_decoded = jax.tree_util.tree_map(compute_metrics_fast, train_data_target_decoded, predicted_target_train_decoded)
        # mean_train_metrics_decoded = compute_mean_metrics(train_metrics_decoded, prefix="decoded_train_")

        # train_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, predicted_target_train_decoded, train_deg_dict)
        # train_deg_target_decoded = jax.tree_util.tree_map(get_mask, train_data_target_decoded, test_deg_dict)

        predicted_target_test = jax.tree_util.tree_map(model.transport, test_data_source_tmp, test_data_conditions_tmp)
        # test_metrics = jax.tree_util.tree_map(compute_metrics_fast, test_data_target_tmp, predicted_target_test)
        # mean_test_metrics = compute_mean_metrics(test_metrics, prefix="test_")

        predicted_target_test_decoded = jax.tree_util.tree_map(reconstruct_data_fn, predicted_target_test)
        test_metrics_decoded = jax.tree_util.tree_map(
            compute_metrics_fast, test_data_target_decoded_tmp, predicted_target_test_decoded
        )
        mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

        # test_deg_target_decoded_predicted = jax.tree_util.tree_map(
        #    get_mask, predicted_target_test_decoded, test_deg_dict_tmp
        # )
        # test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded_tmp, test_deg_dict_tmp)
        # deg_test_metrics_encoded = jax.tree_util.tree_map(
        #    compute_metrics_fast, test_deg_target_decoded, test_deg_target_decoded_predicted
        # )
        # deg_mean_test_metrics_encoded = compute_mean_metrics(deg_test_metrics_encoded, prefix="deg_test_")

        predicted_target_ood = jax.tree_util.tree_map(model.transport, ood_data_source, ood_data_conditions)
        ood_metrics = jax.tree_util.tree_map(compute_metrics_fast, ood_data_target, predicted_target_ood)
        mean_ood_metrics = compute_mean_metrics(ood_metrics, prefix="ood_")

        predicted_target_ood_decoded = jax.tree_util.tree_map(reconstruct_data_fn, predicted_target_ood)
        ood_metrics_decoded = jax.tree_util.tree_map(
            compute_metrics_fast, ood_data_target_decoded, predicted_target_ood_decoded
        )
        mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

        # ood_deg_target_decoded_predicted = jax.tree_util.tree_map(
        #    get_mask, predicted_target_ood_decoded, ood_deg_dict
        # )
        # ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)
        # deg_ood_metrics_encoded = jax.tree_util.tree_map(
        #    compute_metrics_fast, ood_deg_target_decoded, ood_deg_target_decoded_predicted
        # )
        # deg_mean_ood_metrics_encoded = compute_mean_metrics(deg_ood_metrics_encoded, prefix="deg_ood_")
        print(mean_test_metrics_decoded, mean_ood_metrics_decoded)

100%|██████████| 100000/100000 [12:14<00:00, 136.07it/s]


In [18]:
predicted_target_ood = jax.tree_util.tree_map(model.transport, ood_data_source, ood_data_conditions)
ood_metrics = jax.tree_util.tree_map(compute_metrics_fast, ood_data_target, predicted_target_ood)
mean_ood_metrics = compute_mean_metrics(ood_metrics, prefix="ood_")

predicted_target_ood_decoded = jax.tree_util.tree_map(reconstruct_data_fn, predicted_target_ood)
ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, ood_data_target_decoded, predicted_target_ood_decoded
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [19]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': -0.6959954937548297,
 'decoded_ood_e_distance': 51.647755503406145,
 'decoded_ood_mmd_distance': 0.1288715211311746}

In [13]:
ood_metrics_decoded

{'A549_A-366_10.0': {'r_squared': -1.892633773604254,
  'e_distance': 88.08870271998903,
  'mmd_distance': 0.22087055997840274},
 'A549_A-366_100.0': {'r_squared': -1.892685998299453,
  'e_distance': 88.0902931052567,
  'mmd_distance': 0.22086954061269737},
 'A549_A-366_10000.0': {'r_squared': -1.8928958562553757,
  'e_distance': 88.09668385847338,
  'mmd_distance': 0.2208850891576631},
 'A549_Carmofur_10.0': {'r_squared': -1.8927973642599647,
  'e_distance': 88.09368450540867,
  'mmd_distance': 0.22088357537043088},
 'A549_Carmofur_100.0': {'r_squared': -1.8930694173989373,
  'e_distance': 88.10196927567513,
  'mmd_distance': 0.22089576104956812},
 'A549_Disulfiram__10.0': {'r_squared': -1.8725561412197287,
  'e_distance': 87.47728324135454,
  'mmd_distance': 0.2199586645957762},
 'A549_Disulfiram__1000.0': {'r_squared': -1.8729995933796886,
  'e_distance': 87.49078758452193,
  'mmd_distance': 0.21997403666989754},
 'A549_Disulfiram__10000.0': {'r_squared': -1.8731945827348837,
  'e_d

In [15]:
len([el for el in test_data_source.keys() if el.startswith("K562")])

353

In [16]:
len([el for el in test_data_source.keys() if el.startswith("MCF7")])

619

In [17]:
len([el for el in test_data_source.keys() if el.startswith("A549")])

303

In [12]:
next(train_loader)

{'src_lin': tensor([[ 4.4150e+00, -5.2224e+00,  5.1038e-01, -1.6458e-01,  3.0354e-01,
           2.9635e-01,  3.3364e-01, -1.4438e-01,  8.4877e-02,  8.2167e-02,
          -7.2705e-01, -4.0932e-02, -1.3082e-01,  5.0285e-01,  4.8850e-01,
          -5.7499e-01, -1.1404e-01,  1.3718e-01, -6.2031e-01,  3.6175e-01,
          -1.6540e-01,  8.9441e-03,  1.2711e-01, -1.1288e+00, -3.6618e-01,
           5.1373e-02, -5.3618e-01, -2.3007e-01,  7.8937e-01, -5.9700e-01],
         [ 2.8632e+00, -2.6809e+00,  9.8494e-03, -6.9435e-02,  3.8905e-02,
           4.9173e-02,  4.2219e-01, -4.9947e-01,  1.4549e-01,  2.7623e-01,
          -1.6383e-01, -2.5270e-01, -8.9089e-01, -1.2520e-01, -4.8225e-01,
          -2.9720e-01,  1.2243e+00,  8.9002e-02,  1.2691e-01,  6.6709e-01,
          -4.5050e-01,  9.0263e-02,  1.8154e-01,  7.7614e-03, -3.0231e-01,
           1.5803e-01,  9.7377e-01,  4.8168e-01,  3.7946e-01, -2.7502e-02],
         [ 3.3654e+00, -4.1505e+00, -1.2386e-01, -3.2441e-01,  4.2856e-01,
          -6

In [13]:
next(train_loader)

{'src_lin': tensor([[ 3.5087e+00,  3.2612e+00,  5.5268e-02,  4.9694e-01,  3.6385e-01,
          -3.4814e-01, -1.7978e-02,  4.0228e-01,  2.9608e-01, -6.0487e-01,
           1.3812e-01,  2.5441e-01, -3.7777e-01,  2.1129e-01, -1.6405e-01,
           6.0328e-01,  1.9044e-01,  1.5391e-01,  4.2021e-01, -2.0974e-01,
           2.4508e-01, -7.2720e-01, -1.0311e-01,  1.1180e+00,  8.4735e-01,
          -6.9950e-01,  3.1142e-01,  1.2125e-01,  1.5972e-01, -9.9431e-02],
         [ 3.6379e+00,  3.9363e+00, -4.0290e-01, -1.4623e-01, -1.7316e-02,
          -8.0239e-01, -8.0674e-01,  4.0685e-01, -6.8158e-02,  4.0746e-01,
          -5.1724e-01, -1.3895e-01, -5.5646e-01,  2.1886e-01, -3.3274e-01,
           9.4339e-02,  5.1790e-02,  2.9421e-01, -4.2263e-01, -6.8195e-01,
           3.4036e-02, -6.9245e-01, -6.5575e-03,  1.0188e+00,  7.6734e-01,
          -1.0754e-01,  3.2240e-01,  4.9183e-01, -3.2336e-01,  1.1698e-01],
         [ 2.5191e+00,  2.6949e+00,  1.5541e-01, -3.4100e-02,  3.8520e-01,
           6